# Libraries
---

In [239]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [240]:
df = pd.read_csv('https://cocl.us/concrete_data')
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [241]:
df.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

In [242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


## Let's check for missing values

In [243]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Cement,1030.0,281.167864,104.506364,102.00,192.375,272.900,350.000,540.0
Blast Furnace Slag,1030.0,73.895825,86.279342,0.00,0.000,22.000,142.950,359.4
Fly Ash,1030.0,54.188350,63.997004,0.00,0.000,0.000,118.300,200.1
Water,1030.0,181.567282,21.354219,121.80,164.900,185.000,192.000,247.0
Superplasticizer,1030.0,6.204660,5.973841,0.00,0.000,6.400,10.200,32.2
Coarse Aggregate,1030.0,972.918932,77.753954,801.00,932.000,968.000,1029.400,1145.0
Fine Aggregate,1030.0,773.580485,80.175980,594.00,730.950,779.500,824.000,992.6
Age,1030.0,45.662136,63.169912,1.00,7.000,28.000,56.000,365.0
Strength,1030.0,35.817961,16.705742,2.33,23.710,34.445,46.135,82.6


In [244]:
df.isnull().sum()

,0
Cement,0
Blast Furnace Slag,0
Fly Ash,0
Water,0
Superplasticizer,0
Coarse Aggregate,0
Fine Aggregate,0
Age,0
Strength,0


The data looks very clean and is ready to be used to build our model.

In [245]:
concrete_data_columns = df.columns

predictors = df.drop(columns=['Strength'])
target = df['Strength']

In [246]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [247]:
target.head()

,Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30


## Data Normalisation

In [248]:
predictors = (predictors - predictors.mean()) / predictors.std()
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [249]:
n_cols = predictors.shape[1]

## Model training and evaluation

In [250]:
def regression_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layer, each of 10 hidden units.

In [251]:
model = regression_model()
model

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<Sequential name=sequential_560, built=True>

In [252]:
model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=2)

Epoch 1/50
23/23 - 1s - 54ms/step - loss: 1743.9802 - val_loss: 1254.7000
Epoch 2/50
23/23 - 0s - 7ms/step - loss: 1725.5850 - val_loss: 1242.5167
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 1707.8589 - val_loss: 1230.0031
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 1690.2991 - val_loss: 1217.3479
Epoch 5/50
23/23 - 0s - 4ms/step - loss: 1672.9135 - val_loss: 1204.4087
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 1655.2887 - val_loss: 1191.6746
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1638.0590 - val_loss: 1177.9160
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 1620.4125 - val_loss: 1163.9250
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 1602.5040 - val_loss: 1149.6256
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 1584.4440 - val_loss: 1134.4756
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 1565.6862 - val_loss: 1118.7740
Epoch 12/50
23/23 - 0s - 4ms/step - loss: 1546.5251 - val_loss: 1102.4004
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 1526.7260 - val_loss: 1085.4260
Epoch 14/50
23/23 - 0s - 4ms/step - loss: 1506

## Split data and validation
---

In [253]:
mse_scores = []
for _ in range(50):
    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3, random_state=None)

    # Build and train model
    model = regression_model()
    model.fit(predictors_train, target_train, epochs=50, verbose=0)

    # Evaluate model
    targe_pred = model.predict(predictors_test).flatten()
    mse = mean_squared_error(target_test, targe_pred)
    mse_scores.append(mse)

mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

print(f"Mean MSE over 50 runs: {mean_mse:.4f}")
print(f"Standard Deviation of MSE: {std_mse:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Mean MSE over 50 runs: 362.5396
Standard Deviation of MSE: 115.0953


|Experiment|Mean MSE|Standard MSE Deviation|
|-----------|----------|------------------------|
|Without Normalisation|396.6283|402.3144|
|With Normalisation|362.5396|115.0953|

---
here, we can clearly see that without normalisation, the model didn't perform well